# 09 Training - DS-CNN Model with Spectrogram  
#### Author: Subhojeet Pramanik
#### Cognibit Solutions LLP

Derived from https://arxiv.org/pdf/1711.07128.pdf

DS-CNN Model

In [1]:
import sys
import os
import tensorflow as tf

sys.path.append("../libs")
from classification import input_data
from classification import models
from classification import trainer
from classification import freeze

/home/metal_geek/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/metal_geek/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Change the data folder to use the required data folder.

### Flags

In [2]:
flags=tf.app.flags

In [3]:
flags=tf.app.flags
#Important Directories
flags.DEFINE_string('data_dir','../data/raw','Train Data Folder')
flags.DEFINE_string('summaries_dir','../summaries','Summaries Folder')
flags.DEFINE_string('train_dir','../logs&checkpoint','Directory to write event logs and checkpoint')
flags.DEFINE_string('models_dir','../models','Models Folder')
#Task Specific Parameters
flags.DEFINE_string('wanted_words','yes,no,up,down,left,right,on,off,stop,go','Wanted Words')
flags.DEFINE_float('validation_percentage',10,'Validation Percentage')
flags.DEFINE_float('testing_percentage',10,'Testing Percentage')
flags.DEFINE_integer('sample_rate',16000,'Sample Rate')
flags.DEFINE_integer('clip_duration_ms',1000,'Clip Duration in ms')
flags.DEFINE_float('window_size_ms',20,'How long each spectogram timeslice is')
flags.DEFINE_float('window_stride_ms',10.0,'How far to move in time between frequency windows.')
flags.DEFINE_integer('dct_coefficient_count',257,'How many bins to use for the MFCC fingerprint')
flags.DEFINE_float('time_shift_ms',100.0,'Range to randomly shift the training audio by in time.')

FLAGS=flags.FLAGS

### Variables

In [4]:
model_architecture='ds_cnn_spec'
start_checkpoint='../logs&checkpoint/ds_cnn_spec/ckpt-20000'
logging_interval=10
eval_step_interval=500
save_step_interval=2000
silence_percentage=10.0
unknown_percentage=12.0
background_frequency=0.8
background_volume=0.2
learning_rate='0.0005,0.0001' #Always seperated by comma, trains with each of the learning rate for the given number of iterations
train_steps='10000,20000' #Declare  the training steps for which the learning rates will be used
batch_size=100
model_size_info=[6 ,276 ,10 ,4 ,2 ,1 ,276 ,3 ,3 ,2 ,2 ,276 ,3 ,3 ,1 ,1 ,276 ,3 ,3 ,1 ,1 ,276 ,3 ,3 ,1 ,1 ,276 ,3 ,3 ,1 ,1 ]

### Initialise the get_train_data() get_val_data() and get_test_data() Function

In [5]:
train_dir=os.path.join(FLAGS.data_dir,'train','audio')

In [6]:
model_settings = models.prepare_model_settings(
      len(input_data.prepare_words_list(FLAGS.wanted_words.split(','))),
      FLAGS.sample_rate, FLAGS.clip_duration_ms, FLAGS.window_size_ms,
      FLAGS.window_stride_ms, FLAGS.dct_coefficient_count)
audio_processor = input_data.AudioProcessor(
      train_dir, silence_percentage, unknown_percentage,
      FLAGS.wanted_words.split(','), FLAGS.validation_percentage,
      FLAGS.testing_percentage, model_settings,use_silence_folder=True,use_spectrogram=True)

../data/raw/train/audio/*/*.wav
../data/raw/train/audio/_background_noise_/pink_noise.wav
../data/raw/train/audio/_background_noise_/doing_the_dishes.wav
../data/raw/train/audio/_background_noise_/white_noise.wav
../data/raw/train/audio/_background_noise_/dude_miaowing.wav
../data/raw/train/audio/_background_noise_/running_tap.wav
../data/raw/train/audio/_background_noise_/exercise_bike.wav
Tensor("AudioSpectrogram:0", shape=(?, 99, 257), dtype=float32)


In [7]:
def get_train_data(args):
    sess=args
    time_shift_samples = int((FLAGS.time_shift_ms * FLAGS.sample_rate) / 1000)
    train_fingerprints, train_ground_truth = audio_processor.get_data(
        batch_size, 0, model_settings,background_frequency,
        background_volume, time_shift_samples, 'training', sess)
    return train_fingerprints,train_ground_truth

In [8]:
sess=tf.InteractiveSession()

In [9]:
def get_val_data(args):
    '''
    Input: (sess,offset)
    '''
    sess,i=args
    validation_fingerprints, validation_ground_truth = (
            audio_processor.get_data(batch_size, i, model_settings, 0.0,
                                     0.0, 0, 'validation', sess))
    return validation_fingerprints,validation_ground_truth

In [10]:
def get_test_data(args):
    '''
    Input: (sess,offset)
    '''
    sess,i=args
    test_fingerprints, test_ground_truth = audio_processor.get_data(
        batch_size, i, model_settings, 0.0, 0.0, 0, 'testing', sess)
    return test_fingerprints,test_ground_truth

### Training

In [11]:
def main(_):
    sess=tf.InteractiveSession()
    # Placeholders
    fingerprint_size = model_settings['fingerprint_size']
    label_count = model_settings['label_count']
    fingerprint_input = tf.placeholder(
      tf.float32, [None, fingerprint_size], name='fingerprint_input')
    ground_truth_input = tf.placeholder(
      tf.float32, [None, label_count], name='groundtruth_input')
    set_size = audio_processor.set_size('validation')
    label_count = model_settings['label_count']
    
    # Create Model
    
    logits, dropout_prob = models.create_model(
      fingerprint_input,
      model_settings,
      model_architecture,
      model_size_info=model_size_info,
      is_training=True)
    #Start Training
    extra_args=(dropout_prob,label_count,batch_size,set_size)
    trainer.train(sess,logits,fingerprint_input,ground_truth_input,get_train_data,
                  get_val_data,train_steps,learning_rate,eval_step_interval, logging_interval=logging_interval,
                  start_checkpoint=start_checkpoint,checkpoint_interval=save_step_interval,
                  model_name=model_architecture,train_dir=FLAGS.train_dir,
                  summaries_dir=FLAGS.summaries_dir,args=extra_args)

In [12]:
tf.app.run(main=main)

Instructions for updating:
Please switch to tf.train.get_or_create_global_step
INFO:tensorflow:Training from step: 1 
INFO:tensorflow:Step #10: rate 0.000500, accuracy 8.0%, cross entropy 2.482563
INFO:tensorflow:Step #20: rate 0.000500, accuracy 15.0%, cross entropy 2.416008


KeyboardInterrupt: 

### Freeze

In [ ]:
save_checkpoint='../logs&checkpoint/ds_cnn/ckpt-50000'

In [ ]:
save_path=os.path.join(FLAGS.models_dir,model_architecture,'%s.pb'%os.path.basename(save_checkpoint))
freeze.freeze_graph(FLAGS,model_architecture,save_checkpoint,save_path,model_size_info=model_size_info)

In [ ]:
save_path=os.path.join(FLAGS.models_dir,model_architecture,'%s-batched.pb'%os.path.basename(save_checkpoint))
freeze.freeze_graph(FLAGS,model_architecture,save_checkpoint,save_path,batched=True,model_size_info=model_size_info)